**BBC - Liverpool**

In [ ]:
##BBC

In [ ]:
import time
import re
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
edge_options.add_argument("--headless") 
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

url = "https://www.bbc.co.uk/sport/football/teams/liverpool"
driver.get(url)

time.sleep(5)
html = driver.page_source
driver.quit()

soup = BeautifulSoup(html, "html.parser")
articles = soup.find_all("article", class_="ssrcss-1epmku9-ContentPost e6wdqbx1")

news_data = []

for article in articles:
    date_span = article.find("span")
    if date_span:
        raw_date = date_span.get_text(separator=" ", strip=True)
        
        time_matches = re.findall(r"(\d{1,2}:\d{2})", raw_date)
        if time_matches:
            time_text = time_matches[0]
        else:
            time_text = "00:00"
        
        date_part = re.sub(r"\d{1,2}:\d{2}", "", raw_date).strip()
        tokens = date_part.split()
        if len(tokens) >= 2:
            extracted_date_str = " ".join(tokens[:2])
            try:
                date_obj = datetime.strptime(extracted_date_str, "%d %B")
                date_text = date_obj.replace(year=2025).strftime("%#m/%#d/%Y")
            except Exception as e:
                date_text = "3/17/2025"
        else:
            date_text = "3/17/2025"
    else:
        date_text = "3/17/2025"
        time_text = "00:00"
    
    headline_tag = article.find("span", class_="ssrcss-189b1h2-HeadlineWrap e14e9ror2")
    if headline_tag:
        headline_text = headline_tag.get_text(strip=True)
    else:
        headline_text = "No Headline"
    
    paragraphs = article.find_all("p", class_=re.compile("Paragraph"))
    if paragraphs:
        body_text = " ".join(p.get_text(strip=True) for p in paragraphs)
    else:
        body_text = "No Content"
    
    news_data.append([date_text, time_text, headline_text, body_text])

df = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "Body Context"])

print(df.head())
df.to_csv("Test.csv", index=False)

        Date   Time                                           Headline  \
0  3/17/2025  15:14  Is Salah's cup finals record his Achilles heel...   
1  3/17/2025  15:00  Liverpool 'can suffer' final defeat - Jamespub...   
2  3/17/2025  13:55  'We want to bounce back, and we will'published...   
3  3/17/2025  12:06  'Lack of quality depth must be addressed'publi...   
4  3/17/2025  09:55  'We played into their hands'published at 09:55...   

                                        Body Context  
0  Alongside his team-mates, it was a difficult d...  
1  Former Liverpool goalkeeper David James says L...  
2  Liverpool captain Virgil van Dijk on Instagram...  
3  In the space of a week, the Arne Slot machine ...  
4  Andy Robertson says Liverpool "didn't use the ...  


In [ ]:
edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
edge_options.add_argument("--headless")
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

all_news_data = []

for page in range(1, 41):
    if page == 1:
        url = "https://www.bbc.co.uk/sport/football/teams/liverpool"
    else:
        url = f"https://www.bbc.co.uk/sport/football/teams/liverpool?page={page}"

    print(f"Scraping Page {page}: {url}")
    
    driver.get(url)
    time.sleep(3)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    articles = soup.find_all("article", class_="ssrcss-1epmku9-ContentPost e6wdqbx1")

    for article in articles:
        date_span = article.find("span")
        if date_span:
            raw_date = date_span.get_text(separator=" ", strip=True)
            
            time_matches = re.findall(r"(\d{1,2}:\d{2})", raw_date)
            if time_matches:
                time_text = time_matches[0]
            else:
                time_text = "00:00"
            
            date_part = re.sub(r"\d{1,2}:\d{2}", "", raw_date).strip()
            tokens = date_part.split()
            if len(tokens) >= 2:
                extracted_date_str = " ".join(tokens[:2])
                try:
                    date_obj = datetime.strptime(extracted_date_str, "%d %B")
                    date_text = date_obj.replace(year=2025).strftime("%#m/%#d/%Y")
                except Exception:
                    date_text = "3/17/2025"
            else:
                date_text = "3/17/2025"
        else:
            date_text = "3/17/2025"
            time_text = "00:00"
        
        headline_tag = article.find("span", class_="ssrcss-189b1h2-HeadlineWrap e14e9ror2")
        if headline_tag:
            headline_text = headline_tag.get_text(strip=True)
            lower_headline = headline_text.lower()
            idx = lower_headline.find("published at")
            if idx != -1:
                headline_text = headline_text[:idx].strip()
        else:
            headline_text = "No Headline"
        
        paragraphs = article.find_all("p", class_=re.compile("Paragraph"))
        if paragraphs:
            body_text = " ".join(p.get_text(strip=True) for p in paragraphs)
        else:
            body_text = "No Content"
        
        all_news_data.append([date_text, time_text, headline_text, body_text])

driver.quit()
df = pd.DataFrame(all_news_data, columns=["Date", "Time", "Headline", "Body Context"])
df.to_csv("BBC_AllPages.csv", index=False)
print("Scraping complete! Data saved to AllPages.csv")

Scraping Page 1: https://www.bbc.co.uk/sport/football/teams/liverpool
Scraping Page 2: https://www.bbc.co.uk/sport/football/teams/liverpool?page=2
Scraping Page 3: https://www.bbc.co.uk/sport/football/teams/liverpool?page=3
Scraping Page 4: https://www.bbc.co.uk/sport/football/teams/liverpool?page=4
Scraping Page 5: https://www.bbc.co.uk/sport/football/teams/liverpool?page=5
Scraping Page 6: https://www.bbc.co.uk/sport/football/teams/liverpool?page=6
Scraping Page 7: https://www.bbc.co.uk/sport/football/teams/liverpool?page=7
Scraping Page 8: https://www.bbc.co.uk/sport/football/teams/liverpool?page=8
Scraping Page 9: https://www.bbc.co.uk/sport/football/teams/liverpool?page=9
Scraping Page 10: https://www.bbc.co.uk/sport/football/teams/liverpool?page=10
Scraping Page 11: https://www.bbc.co.uk/sport/football/teams/liverpool?page=11
Scraping Page 12: https://www.bbc.co.uk/sport/football/teams/liverpool?page=12
Scraping Page 13: https://www.bbc.co.uk/sport/football/teams/liverpool?page=1

In [ ]:
##SkySport

In [ ]:
import time
import re
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup


edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

url = "https://www.skysports.com/football/live-blog/11669/13025501/liverpool-transfer-news-rumours-and-gossip-live-updates-and-latest-on-deals-signings-loans-and-contracts"
driver.get(url)

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ncpost-list-post")))

def click_load_more():
    while True:
        try:
            load_more_button = driver.find_element(By.CLASS_NAME, "ui-liveblog-button--load-more")
            
            if load_more_button.is_displayed():
                print("✅ Load More button found! Clicking...")

                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button)
                time.sleep(1) 
                driver.execute_script("arguments[0].click();", load_more_button)
                time.sleep(5)
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ncpost-list-post")))

            else:
                print("❌ Load More button no longer visible.")
                break
        
        except Exception as e:
            print(f"❌ Load More button NOT found or all content loaded: {e}")
            break

click_load_more()

html = driver.page_source

driver.quit()
soup = BeautifulSoup(html, "html.parser")
articles = soup.select("div.ncpost-list-post")

news_data = []

for article in articles:
    headline_tag = article.find("h2", class_="ncpost-title")
    headline_text = headline_tag.get_text(strip=True) if headline_tag else "No Headline"

    time_tag = article.find("time", class_="ncpost-timestamp")
    if time_tag and time_tag.has_attr("datetime"):
        datetime_str = time_tag["datetime"]
        date_obj = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S.%fZ")

        try:
            date_text = date_obj.strftime("%#m/%#d/%Y")
        except ValueError:
            date_text = date_obj.strftime("%-m/%-d/%Y")
            
        time_text = date_obj.strftime("%H:%M")
    else:
        date_text, time_text = "Unknown", "00:00"

    paragraphs = article.select("div.ncpost-content p")
    body_text = " ".join(p.get_text(strip=True) for p in paragraphs) if paragraphs else "No Content"

    news_data.append([date_text, time_text, headline_text, body_text])

dfs = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "Body Context"])

print(dfs.head())
dfs.to_csv("skysports_liverpool_full.csv", index=False)
print("✅ Scraping completed successfully!")

✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More 

In [ ]:
import time  
from selenium import webdriver  
from selenium.webdriver.edge.service import Service as EdgeService  
from selenium.webdriver.edge.options import Options as EdgeOptions  
from selenium.webdriver.common.by import By  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC  

edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
edge_options.add_argument("--headless")
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

url = "https://www.skysports.com/football/live-blog/11669/13025501/liverpool-transfer-news-rumours-and-gossip-live-updates-and-latest-on-deals-signings-loans-and-contracts"
driver.get(url)
time.sleep(5) 

try:
    load_more_button = driver.find_element(By.CLASS_NAME, "ui-liveblog-button--load-more")
    print("✅ Load More button found!")
except:
    print("❌ Load More button NOT found!")

✅ Load More button found!


In [ ]:
import time  
from selenium import webdriver  
from selenium.webdriver.edge.service import Service as EdgeService  
from selenium.webdriver.edge.options import Options as EdgeOptions  
from selenium.webdriver.common.by import By  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC  

edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
edge_options.add_argument("--headless")
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

url = "https://www.skysports.com/football/live-blog/11669/13025501/liverpool-transfer-news-rumours-and-gossip-live-updates-and-latest-on-deals-signings-loans-and-contracts"
driver.get(url)

time.sleep(5)

if load_more_button:
    driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
    time.sleep(2)
    load_more_button.click()
    print("✅ Load More button clicked after scrolling.")

NoSuchElementException: Message: no such element: element not found
  (Session info: MicrosoftEdge=134.0.3124.72); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60E82E735+13397]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF60EABB1B4+2060276]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF60EA12338+1368440]
	(No symbol) [0x00007FF60E5CBC86]
	(No symbol) [0x00007FF60E5CAB7F]
	(No symbol) [0x00007FF60E5CC321]
	(No symbol) [0x00007FF60E5C065E]
	(No symbol) [0x00007FF60E5CC352]
	(No symbol) [0x00007FF60E5C065E]
	(No symbol) [0x00007FF60E5BEA75]
	(No symbol) [0x00007FF60E5C27DD]
	(No symbol) [0x00007FF60E642574]
	(No symbol) [0x00007FF60E62425A]
	(No symbol) [0x00007FF60E5F964D]
	(No symbol) [0x00007FF60E641C00]
	(No symbol) [0x00007FF60E623FC3]
	(No symbol) [0x00007FF60E5F8B96]
	(No symbol) [0x00007FF60E5F7E50]
	(No

In [ ]:
import time  
from selenium import webdriver  
from selenium.webdriver.edge.service import Service as EdgeService  
from selenium.webdriver.edge.options import Options as EdgeOptions  
from selenium.webdriver.common.by import By  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC  

edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

url = "https://www.skysports.com/football/live-blog/11669/13025501/liverpool-transfer-news-rumours-and-gossip-live-updates-and-latest-on-deals-signings-loans-and-contracts"
driver.get(url)

time.sleep(5)

try:
    load_more_button = driver.find_element(By.CSS_SELECTOR, "button.ui-liveblog-button--load-more")
    print("✅ Load More button found!")

    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button)
    time.sleep(2) 

    driver.execute_script("arguments[0].click();", load_more_button)
    print("✅ Load More button clicked with JavaScript!")

except Exception as e:
    print(f"❌ Load More button NOT found or not clickable: {e}")

✅ Load More button found!
✅ Load More button clicked with JavaScript!


In [ ]:
import time
import re
import pandas as pd
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

url = "https://www.skysports.com/football/live-blog/11669/13025501/liverpool-transfer-news-rumours-and-gossip-live-updates-and-latest-on-deals-signings-loans-and-contracts"
driver.get(url)

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ncpost-list-post")))

try:
    load_more_button = driver.find_element(By.CLASS_NAME, "ui-liveblog-button--load-more")
    if load_more_button.is_displayed():
        print("✅ Load More button found! Clicking once for testing...")

        driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        time.sleep(1)
        
        driver.execute_script("arguments[0].click();", load_more_button)

        time.sleep(5)

        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ncpost-list-post")))

        print("✅ Load More button clicked successfully!")

    else:
        print("❌ Load More button found but not visible.")
except Exception as e:
    print(f"❌ Load More button NOT found or not clickable: {e}")

html = driver.page_source
driver.quit()
soup = BeautifulSoup(html, "html.parser")
articles = soup.select("div.ncpost-list-post")

news_data = []

for article in articles:
    headline_tag = article.find("h2", class_="ncpost-title")
    headline_text = headline_tag.get_text(strip=True) if headline_tag else "No Headline"

    time_tag = article.find("time", class_="ncpost-timestamp")
    if time_tag and time_tag.has_attr("datetime"):
        datetime_str = time_tag["datetime"]
        date_obj = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S.%fZ")

        try:
            date_text = date_obj.strftime("%#m/%#d/%Y")
        except ValueError:
            date_text = date_obj.strftime("%-m/%-d/%Y")
            
        time_text = date_obj.strftime("%H:%M")
    else:
        date_text, time_text = "Unknown", "00:00"

    paragraphs = article.select("div.ncpost-content p")
    body_text = " ".join(p.get_text(strip=True) for p in paragraphs) if paragraphs else "No Content"
    news_data.append([date_text, time_text, headline_text, body_text])

dfs = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "Body Context"])
print(dfs.head())
dfs.to_csv("skysports_liverpool_test.csv", index=False)
print("✅ Scraping completed successfully!")

✅ Load More button found! Clicking once for testing...
✅ Load More button clicked successfully!
        Date   Time                                           Headline  \
0  3/17/2025  14:33  Merson calls for perspective after 'bad week' ...   
1  3/17/2025  11:59  Ref Watch: Carabao Cup Final - Big decisions a...   
2  3/17/2025  11:08  Van Dijk: There'll be news on my Liverpool fut...   
3  3/16/2025  20:26           LISTEN: Carabao Cup final review podcast   
4  3/16/2025  19:56       Carra: Worst cup final performance for years   

                                        Body Context  
0  Paul Merson says Liverpool have had a bad week...  
1  The Ref Watch team cast their eye over the big...  
2  LiverpoolcaptainVirgil van Dijksays there will...  
3  Mark Chapman is joined by Shay Given, Les Ferd...  
4  Sky Sports' Jamie Carragher reflecting on Live...  
✅ Scraping completed successfully!


In [ ]:
import time  
from selenium import webdriver  
from selenium.webdriver.edge.service import Service as EdgeService  
from selenium.webdriver.edge.options import Options as EdgeOptions  
from selenium.webdriver.common.by import By  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC  

edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
edge_options.add_argument("--headless")
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

url = "https://www.skysports.com/football/live-blog/11669/13025501/liverpool-transfer-news-rumours-and-gossip-live-updates-and-latest-on-deals-signings-loans-and-contracts"
driver.get(url)

time.sleep(5)

iframes = driver.find_elements(By.TAG_NAME, "iframe")
print(f"🔍 Found {len(iframes)} iframes.")

if len(iframes) > 0:
    driver.switch_to.frame(iframes[0])
    print("✅ Switched to iframe.")

🔍 Found 18 iframes.
✅ Switched to iframe.


In [ ]:
##This is Anfield

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime
import unicodedata

edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

def clean_text(text):
    text = text.replace("\u2018", "'").replace("\u2019", "'")
    text = text.replace("\u201C", '"').replace("\u201D", '"') 
    text = text.replace("\u2013", "-").replace("\u2014", "-") 
    text = unicodedata.normalize("NFKD", text)
    return text.strip()

url = "https://www.thisisanfield.com/liverpool-fc-news/"
driver.get(url)

wait = WebDriverWait(driver, 30)
try:
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h2.entry-title a")))
except:
    print("❌ Timeout: Articles not found. Trying alternative methods...")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
articles = soup.select("article")
news_data = []

for article in articles:
    headline_element = article.select_one("h2.entry-title a")
    date_element = article.select_one("time.entry-date, time.published, span.posted-on time")
    
    if headline_element and date_element:
        headline = clean_text(headline_element.text)
        news_url = headline_element["href"]
        date_published = clean_text(date_element.text)
        
        time_published = date_element.get("datetime", "Unknown")
        if time_published != "Unknown":
            time_published = datetime.fromisoformat(time_published.replace("Z", "+00:00")).strftime("%H:%M:%S")
        
        print(f"Extracted: {headline} | Date: {date_published} | Time: {time_published}")
        
        news_data.append([date_published, time_published, headline, news_url])

for i, (_, _, _, news_url) in enumerate(news_data):
    driver.get(news_url)
    time.sleep(2)
    
    article_html = driver.page_source
    article_soup = BeautifulSoup(article_html, "html.parser")
    
    body_elements = article_soup.select("div.elementor-widget-theme-post-content p, div.elementor-widget-theme-post-content h2, div.elementor-widget-theme-post-content blockquote")
    body_content = "\n".join([clean_text(element.text) for element in body_elements if element.text.strip()])
    
    news_data[i].append(body_content)

driver.quit()
dfa = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "URL", "Body Context"])
dfa.to_csv("thisisanfield_news.csv", index=False, encoding="utf-8-sig")
print("✅ Scraping completed successfully! Data saved as 'thisisanfield_news.csv'")

Extracted: Jeremie Frimpong transfer links explained after meeting with Liverpool trio | Date: Mar 18, 2025 | Time: 15:48:26
Extracted: Julian Alvarez to Liverpool transfer update - Atletico Madrid respond on 'official offer' | Date: Mar 18, 2025 | Time: 13:36:56
Extracted: Liverpool's youngest-ever player now part of YouTube league aged 28 | Date: Mar 18, 2025 | Time: 12:47:40
Extracted: Conor Bradley injury update in mixed news for Liverpool - "He's frustrated" | Date: Mar 18, 2025 | Time: 11:19:53
Extracted: Robbie Fowler says "unbelievable statement" sums up Arne Slot at Liverpool | Date: Mar 18, 2025 | Time: 10:46:09
Extracted: Federico Chiesa given advantage no other Liverpool player has this month | Date: Mar 18, 2025 | Time: 10:07:31
Extracted: Luis Diaz 'transfer interest' from Saudi Arabia - "concrete possibility" to leave | Date: Mar 18, 2025 | Time: 09:45:04
Extracted: Liverpool FC international break: 20 players called up for March fixtures | Date: Mar 18, 2025 | Time: 09:

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random

def clean_text(text):
    return text.strip()

csv_file = "thisisanfield_news_backup_link.csv"
df = pd.read_csv(csv_file)

if "Body Context" not in df.columns:
    df["Body Context"] = "No Content"

for index, row in df.iterrows():
    url = row["URL"]
    print(f"🔍 Scraping: {url}")

    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            print(f"⚠️ Error {response.status_code}: Skipping {url}")
            df.at[index, "Body Context"] = f"Error {response.status_code}"
            continue

        soup = BeautifulSoup(response.text, "html.parser")

        body_elements = soup.select("div.elementor-widget-theme-post-content p, div.entry-content p")
        body_content = "\n".join([clean_text(element.text) for element in body_elements if element.text.strip()])

        if not body_content:
            print(f"⚠️ Warning: No body content found for {url}")
            df.at[index, "Body Context"] = "No Content"
        else:
            df.at[index, "Body Context"] = body_content[:500]

        print(f"✅ Successfully Scraped Row {index + 1}:")
        print(f"🔎 First 10 words: {' '.join(body_content.split()[:10])}...\n")

        time.sleep(random.uniform(2, 5))

    except Exception as e:
        print(f"❌ Error: {e}")
        df.at[index, "Body Context"] = "Scraping Error"

    if (index + 1) % 10 == 0:
        df.to_csv("thisisanfield_news_temp.csv", index=False, encoding="utf-8-sig")
        print(f"💾 Temporary save at row {index + 1}")

df.to_csv("thisisanfield_news_updated.csv", index=False, encoding="utf-8-sig")
print("🎉 Scraping Completed! Data saved in 'thisisanfield_news_updated.csv'")

🔍 Scraping: https://www.thisisanfield.com/2025/03/jeremie-frimpong-transfer-links-explained-after-meeting-with-liverpool-trio/
✅ Successfully Scraped Row 1:
🔎 First 10 words: Bayer Leverkusen right-back Jeremie Frimpong has been linked with a...

🔍 Scraping: https://www.thisisanfield.com/2025/03/julian-alvarez-to-liverpool-transfer-update-atletico-madrid-respond-on-official-offer/
✅ Successfully Scraped Row 2:
🔎 First 10 words: Julian Alvarez was subject of interest from Liverpool last summer,...

🔍 Scraping: https://www.thisisanfield.com/2025/03/liverpools-youngest-ever-player-now-part-of-youtube-league-aged-28/
✅ Successfully Scraped Row 3:
🔎 First 10 words: The record for Liverpool’s youngest-ever player has not been broken...

🔍 Scraping: https://www.thisisanfield.com/2025/03/conor-bradley-injury-update-in-mixed-news-for-liverpool-hes-frustrated/
✅ Successfully Scraped Row 4:
🔎 First 10 words: Conor Bradley is not expected to make an imminent return...

🔍 Scraping: https://www.this